In [1]:
# Install these packages if running from colab
!pip install tensorflow-datasets --quiet
!pip install pydot --quiet
!pip install transformers --quiet

# install huggingface datasets
!pip install datasets --quiet

! pip install rouge-score nltk --quiet
! pip install huggingface_hub --quiet

!pip install sentencepiece --quiet

     |████████████████████████████████| 5.5 MB 4.7 MB/s 
     |████████████████████████████████| 163 kB 82.8 MB/s 
     |████████████████████████████████| 7.6 MB 78.8 MB/s 
     |████████████████████████████████| 441 kB 4.7 MB/s 
     |████████████████████████████████| 212 kB 89.9 MB/s 
     |████████████████████████████████| 95 kB 5.7 MB/s 
     |████████████████████████████████| 115 kB 87.4 MB/s 
     |████████████████████████████████| 127 kB 87.2 MB/s 
     |████████████████████████████████| 115 kB 91.0 MB/s 
     |████████████████████████████████| 1.3 MB 4.7 MB/s 


In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re

#let's make longer output readable without scrolling
from pprint import pprint

# the toxic parallel dataset, with rouge metric
from datasets import load_dataset, load_from_disk, load_metric, DatasetDict

In [3]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# define paths
dataset_path = 'drive/MyDrive/Colab Notebooks/w266_project_data'
csv_path = 'drive/MyDrive/Colab Notebooks/w266_project_predictions/'

#### Change these variables as needed for different model and different file name

In [5]:
# change these variables for different models
output_file_name = 't5_large_zsl.csv'
model_checkpoint = "t5-large"

### Import and examine our dataset

In [6]:
# load the dataset
dataset = load_from_disk(dataset_path)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 17789
    })
    test: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 989
    })
    valid: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 988
    })
})

In [8]:
metric = load_metric("rouge")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [9]:
pd.DataFrame(dataset['train'])

,en_toxic_comment,en_neutral_comment
0,u 'd be surprised all the shit u think about w...,You would be supriesd all the things you think...
1,plenty of holocaust deniers actually believe t...,Plenty of Holocaust deniers actually believe t...
2,holy fuck i 'm so going,i'm so going
3,i 've driven by lots of cattle farms but holy ...,i 've driven by lots of cattle farms but this
4,so much better this shit to soft for me,So much better this is to soft for me.
...,...,...
17784,"but , thats exactly what a couple of you idiot...","But, that’s exactly what a couple of you have ..."
17785,he probably assessed the situation and decided...,He probably assessed the situation and decided...
17786,brah im fucked up over here .,Brah I am feeling bad over here
17787,shit with my friends and not being able to tru...,with my friends and not being able to trust th...


## load T5 large pretrain model

In [10]:
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration

model = TFT5ForConditionalGeneration.from_pretrained(model_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [11]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  32899072  
                                                                 
 encoder (TFT5MainLayer)     multiple                  334939648 
                                                                 
 decoder (TFT5MainLayer)     multiple                  435627520 
                                                                 
Total params: 737,668,096
Trainable params: 737,668,096
Non-trainable params: 0
_________________________________________________________________


### Preprocess: Encode detox dataset with T5 tokenizer

In [12]:
## Encode detox train_texts with T5 tokenizer
max_length = 25
#max_label_length = 128

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples['en_toxic_comment']]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['en_neutral_comment'], max_length=max_length, truncation=True, padding=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
# test the preprocess function
preprocess_function(dataset['train'][:2])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


{'input_ids': [[21603, 10, 3, 76, 3, 31, 26, 36, 5597, 66, 8, 3, 7, 10536, 3, 76, 317, 81, 116, 3, 76, 3, 2047, 7, 1], [21603, 10, 2500, 13, 3534, 5133, 2064, 17, 177, 4518, 700, 857, 8, 6139, 3, 7, 10536, 79, 497, 3, 5, 1, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]], 'labels': [[148, 133, 36, 3, 7, 413, 2593, 26, 66, 8, 378, 25, 317, 81, 116, 25, 33, 131, 3823, 132, 5, 1], [28474, 13, 26178, 177, 4518, 700, 857, 8, 378, 79, 497, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

In [14]:
# tokenize the detox dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

### no fine-tuning, zero-shot. Try on one example

In [15]:
input_tokenized = tokenizer(["summarize: " + dataset['test']['en_toxic_comment'][1]], return_tensors="tf").input_ids
summary_ids = model.generate(input_tokenized, num_beams=2, min_length=0, max_length=25)

In [16]:
prediction = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
reference = [dataset['test']['en_neutral_comment'][1]]

In [17]:
print([dataset['test']['en_toxic_comment'][1]])
print(prediction)
print(reference)

['all she has to do is smile and keep her cool with slobbering old wrinkled hubby .']
['all she has to do is smile and keep her cool with slobbering old wrinkled husband .']
['All she has to do is smile and keep her cool with old aged hubby']


In [18]:
rouge_results = metric.compute(predictions=prediction,
                               references=reference)
pprint(rouge_results, compact=True)  

{'rouge1': AggregateScore(low=Score(precision=0.8125, recall=0.8666666666666667, fmeasure=0.8387096774193549), mid=Score(precision=0.8125, recall=0.8666666666666667, fmeasure=0.8387096774193549), high=Score(precision=0.8125, recall=0.8666666666666667, fmeasure=0.8387096774193549)),
 'rouge2': AggregateScore(low=Score(precision=0.7333333333333333, recall=0.7857142857142857, fmeasure=0.7586206896551724), mid=Score(precision=0.7333333333333333, recall=0.7857142857142857, fmeasure=0.7586206896551724), high=Score(precision=0.7333333333333333, recall=0.7857142857142857, fmeasure=0.7586206896551724)),
 'rougeL': AggregateScore(low=Score(precision=0.8125, recall=0.8666666666666667, fmeasure=0.8387096774193549), mid=Score(precision=0.8125, recall=0.8666666666666667, fmeasure=0.8387096774193549), high=Score(precision=0.8125, recall=0.8666666666666667, fmeasure=0.8387096774193549)),
 'rougeLsum': AggregateScore(low=Score(precision=0.8125, recall=0.8666666666666667, fmeasure=0.8387096774193549), m

### zero-shot learning (ZSL) on the test set

In [19]:
test_predictions = []
test_references = []
test_len = len(dataset['test']['en_toxic_comment'])

for i in range(test_len):
  input_tokenized = tokenizer(["summarize: " + dataset['test']['en_toxic_comment'][i]], return_tensors="tf").input_ids
  summary_ids = model.generate(input_tokenized, num_beams=2, min_length=0, max_length=25)
  
  prediction = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  reference = [dataset['test']['en_neutral_comment'][i]]
  
  test_predictions.insert(i, (prediction))
  test_references.insert(i, (reference))

  if i % 50 == 0:
    print('complete', i, '/', test_len)
#print(len(val_references))

complete 0 / 989
complete 50 / 989
complete 100 / 989
complete 150 / 989
complete 200 / 989
complete 250 / 989
complete 300 / 989
complete 350 / 989
complete 400 / 989
complete 450 / 989
complete 500 / 989
complete 550 / 989
complete 600 / 989
complete 650 / 989
complete 700 / 989
complete 750 / 989
complete 800 / 989
complete 850 / 989
complete 900 / 989
complete 950 / 989


In [20]:
# dictionary of lists  
dict = {'test_predictions': test_predictions, 'test_references': test_references}  
       
df = pd.DataFrame(dict) 

In [21]:
# saving the output dataframe to a csv file
# output_file_name = 'bart_large_zsl.csv'
df.to_csv(csv_path + output_file_name, index = False) 

In [27]:
# read data from csv file
df_bart_predictions = pd.read_csv(csv_path + output_file_name)

# display data frame
print(df_bart_predictions[:10])

# display shape
print(df_bart_predictions.shape)

# display columns
print(df_bart_predictions.columns)

                                    test_predictions  \
0             ['the 61 61 view must be hilarious .']   
1  ['all she has to do is smile and keep her cool...   
2  ['can we apply that litmus test to muslims tha...   
3  ['im joining youtube . im joining youtube . im...   
4  ['residents of davis have a collective stick u...   
5  ["i'm being blanked by my best mate silly bitc...   
6  ['this idea of just stacking things into legis...   
7                           ['this shit is crazy .']   
8  ['i seem to remember a certain dotard promisin...   
9  ["she better not do no shit like that ' i woul...   

                                     test_references  
0                    ['The 6161 view must be funny']  
1  ['All she has to do is smile and keep her cool...  
2  ['can we apply that litmus test to muslims tha...  
3                        ["That's it! I'm joining."]  
4    ['The residents of Davis are not so pleasant.']  
5  ['Being blanked by my best mate,. Lol his plea... 

In [23]:
rouge_results = metric.compute(predictions=df_bart_predictions['test_predictions'],
                               references=df_bart_predictions['test_references'])
pprint(rouge_results, compact=True)  

{'rouge1': AggregateScore(low=Score(precision=0.6389682046163707, recall=0.7056904303275536, fmeasure=0.657160884143948), mid=Score(precision=0.6532661028574085, recall=0.7204241342006374, fmeasure=0.6711042296727541), high=Score(precision=0.6691126257272139, recall=0.7370202045425507, fmeasure=0.6858398907801218)),
 'rouge2': AggregateScore(low=Score(precision=0.4865724574635584, recall=0.5396407112898072, fmeasure=0.4993590963467753), mid=Score(precision=0.5039518650992347, recall=0.5582444382989353, fmeasure=0.5168177157776084), high=Score(precision=0.5214989542577694, recall=0.5762292234107945, fmeasure=0.5336277424976579)),
 'rougeL': AggregateScore(low=Score(precision=0.630086133730532, recall=0.6956789563341923, fmeasure=0.6473756823456909), mid=Score(precision=0.6450868460072254, recall=0.7112594024995962, fmeasure=0.6622308116111655), high=Score(precision=0.6611272452332214, recall=0.7273918335357046, fmeasure=0.6773079860633753)),
 'rougeLsum': AggregateScore(low=Score(precis

In [24]:
pd.DataFrame.from_dict(rouge_results)

,rouge1,rouge2,rougeL,rougeLsum
0,"(0.6389682046163707, 0.7056904303275536, 0.657...","(0.4865724574635584, 0.5396407112898072, 0.499...","(0.630086133730532, 0.6956789563341923, 0.6473...","(0.6299065387242784, 0.6960015248870749, 0.648..."
1,"(0.6532661028574085, 0.7204241342006374, 0.671...","(0.5039518650992347, 0.5582444382989353, 0.516...","(0.6450868460072254, 0.7112594024995962, 0.662...","(0.6449068321582778, 0.7112768518366123, 0.662..."
2,"(0.6691126257272139, 0.7370202045425507, 0.685...","(0.5214989542577694, 0.5762292234107945, 0.533...","(0.6611272452332214, 0.7273918335357046, 0.677...","(0.6591169489268941, 0.7273070064349271, 0.676..."


In [25]:
pd.concat({k: pd.DataFrame(v) for k, v in rouge_results.items()})

precision    recall  fmeasure
rouge1    0   0.638968  0.705690  0.657161
          1   0.653266  0.720424  0.671104
          2   0.669113  0.737020  0.685840
rouge2    0   0.486572  0.539641  0.499359
          1   0.503952  0.558244  0.516818
          2   0.521499  0.576229  0.533628
rougeL    0   0.630086  0.695679  0.647376
          1   0.645087  0.711259  0.662231
          2   0.661127  0.727392  0.677308
rougeLsum 0   0.629907  0.696002  0.648241
          1   0.644907  0.711277  0.662225
          2   0.659117  0.727307  0.676245